In [1]:
# Import CVX and numpy libraries
import cvxpy as cvx
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
def generate(n,npoints):
    # Set the limits
    origA = np.full((n+1,n),-5.)
    rangA = np.full((n+1,n), 10.)
    origP = np.full((npoints,n),-5.)
    rangP = np.full((npoints,n), 10.)

    # Construct the anchor points
    a = np.random.rand(n+1,n)
    a = origA + np.multiply(a,rangA)

    # Construct the sensor points
    p = np.random.rand(npoints,n)
    p = origP + np.multiply(p,rangP)

    adjacency = np.full((npoints+n+1,npoints+n+1),-1.)

    if(n > 1):
        minConstr = 3
        maxConstr = 4
        minPointConstr = 2
        minAnchConstr = 2
    else:
        minConstr = 2
        maxConstr = 3
        minPointConstr = 1
        minAnchConstr = 1

    anchList = [i for i in range(n+1)]
    pointList = [i for i in range(npoints)]
    off = n + 1

    for i in range(npoints):
        random.shuffle(anchList)
        random.shuffle(pointList)
        # Set anchor constraints, between 1 and 3
        numAnch = random.randint(minAnchConstr,minConstr)
        for l in range(numAnch):
            adjacency[anchList[l],i+off] = np.linalg.norm(a[anchList[l]] - p[i])
            adjacency[i+off,anchList[l]] = adjacency[anchList[l],i+off]
        print(numAnch)
        # Count number of constraints already set for this point
        countConstr = 0
        for l in range(npoints+off):
            if(adjacency[l,i+off] > 0):
                countConstr = countConstr + 1
        # Set point constraints, between 0 and 3
        if(minConstr - countConstr >= 0):
            numPoint = random.randint(minConstr-countConstr,3)
            count = numPoint
            l = 0
            while(count > 0 and l < len(pointList)):
                if(pointList[l] != i):
                        countConstr = 0
                        for k in range(npoints+off):
                            if(adjacency[k,pointList[l]+off] > 0):
                                countConstr = countConstr + 1
                        if(countConstr < maxConstr):
                            adjacency[pointList[l]+off,i+off] = np.linalg.norm(p[pointList[l]] - p[i])
                            adjacency[i+off,pointList[l]+off] = adjacency[pointList[l]+off,i+off]
                            count = count - 1
                l = l + 1
    return [a, p, adjacency]

In [3]:
def generate_inside_hull(n,npoints):
    # Set the limits
    origA = np.full((n+1,n),-5.)
    rangA = np.full((n+1,n), 10.)
    origP = np.full((npoints,n),-5.)
    rangP = np.full((npoints,n), 10.)
    
    if(n == 1):
        a = np.array([-6, 6])
    elif(n == 2):
        a = np.array([[-6, 15], [-6, -15], [8, 0]])
    elif(n == 3):
        a = np.array([[-6, 15,-10], [-6, -15,-10], [8, 0, -10], [0, 0, 10]])
    # Construct the sensor points
    p = np.random.rand(npoints,n)
    p = origP + np.multiply(p,rangP)

    adjacency = np.full((npoints+n+1,npoints+n+1),-1.)

    if(n > 1):
        minConstr = 3
        maxConstr = 4
        minPointConstr = 2
        minAnchConstr = 2
    else:
        minConstr = 2
        maxConstr = 3
        minPointConstr = 1
        minAnchConstr = 1

    anchList = [i for i in range(n+1)]
    pointList = [i for i in range(npoints)]
    off = n + 1

    for i in range(npoints):
        random.shuffle(anchList)
        random.shuffle(pointList)
        # Set anchor constraints, between 1 and 3
        numAnch = random.randint(minAnchConstr,minConstr)
        for l in range(numAnch):
            adjacency[anchList[l],i+off] = np.linalg.norm(a[anchList[l]] - p[i])
            adjacency[i+off,anchList[l]] = adjacency[anchList[l],i+off]
        print(numAnch)
        # Count number of constraints already set for this point
        countConstr = 0
        for l in range(npoints+off):
            if(adjacency[l,i+off] > 0):
                countConstr = countConstr + 1
        # Set point constraints, between 0 and 3
        if(minConstr - countConstr >= 0):
            numPoint = random.randint(minConstr-countConstr,3)
            count = numPoint
            l = 0
            while(count > 0 and l < len(pointList)):
                if(pointList[l] != i):
                        countConstr = 0
                        for k in range(npoints+off):
                            if(adjacency[k,pointList[l]+off] > 0):
                                countConstr = countConstr + 1
                        if(countConstr < maxConstr):
                            adjacency[pointList[l]+off,i+off] = np.linalg.norm(p[pointList[l]] - p[i])
                            adjacency[i+off,pointList[l]+off] = adjacency[pointList[l]+off,i+off]
                            count = count - 1
                l = l + 1
    return [a, p, adjacency]

In [8]:
n = 2
npoints = 10

(a, p, adjacency) = generate(n, npoints)

# Compute the Euclidian distances to the anchor points
adjSize = len(p) + len(a)
asize = len(a)
d = []

for i in range(adjSize):
    for j in range(adjSize):
        if(j > i and adjacency[i][j] > 0 and i < asize):
            d.append((adjacency[i][j], j - asize, i, True))
        elif(j > i and adjacency[i][j] > 0):
            d.append((adjacency[i][j], i - asize, j - asize, False))

# Construct the CVX variables to minimize
x = [cvx.Variable(n) for i in range(len(p))]

T = n + npoints

# We are working in R^2
            
#This creates a 3x3 semipositive definite matrix which we will 
#use as part of our constraints
z = cvx.Semidef(T)
print a, p, d

eyeConstraint = []
anchorConstraints = []
pointConstraints = []

for i in range(n):
    temp = np.zeros((T,T))
    temp[i][i] = 1
    eyeConstraint.append(temp)
    
temp = np.zeros((T,T))
for i in range(n):
    for j in range(n):
        temp[i][j] = 1
eyeConstraint.append(temp)

for (distance, i, j, truth) in d:
    if truth:
        temp = np.zeros(npoints)
        temp[i] = -1.
        anchorConstraints.append((np.outer(np.append(a[j], temp), np.append(a[j], temp)), distance))
    else:
        tempi = np.zeros(npoints)
        tempj = np.zeros(npoints)
        tempi[i] = 1.
        tempj[j] = 1.
        temp = tempi - tempj
        corner = np.zeros(n)
        temp = np.append(corner, temp)
        pointConstraints.append((np.outer(temp,temp), distance))

#Another empty states list
states = []

cost = cvx.norm(0)

#The four constraints in the SDP relaxation problem
#Note that the last constraint forces z to be SPD
constr = []

for i, mat in enumerate(eyeConstraint):
    if i < len(eyeConstraint) - 1:
        constr.append(cvx.sum_entries(cvx.mul_elemwise(mat, z)) == 1)
    else:
        constr.append(cvx.sum_entries(cvx.mul_elemwise(mat, z)) == n)

for mat in anchorConstraints:
    constr.append(cvx.sum_entries(cvx.mul_elemwise(mat[0], z)) ==  mat[1] ** 2)

for mat in pointConstraints:
    constr.append(cvx.sum_entries(cvx.mul_elemwise(mat[0], z)) ==  mat[1] ** 2)

constr.append(z >> 0)

#Add the constraints and cost function
states.append(cvx.Problem(cvx.Minimize(cost), constr))

#Solve the SDP relaxation problem
prob = sum(states)
prob.solve();    

print('Solution: ' + prob.status)

for i in range(npoints):
    soln1 = z.value.A[0:n, i + n]
    point1 = p[i]
    print("Sensor " + str(i) + " is located at " + str(soln1) + " and the actual value is " + str(point1))

3
2
2
2
2
3
2
2
3
2
[[-1.03530528  4.48975561]
 [ 2.25243113  3.1237451 ]
 [-1.14498481  2.03574217]] [[ 4.19283275 -0.57562836]
 [ 3.72680453 -4.85998454]
 [-1.68017424  1.45928148]
 [-4.26489822  2.93648509]
 [-4.62960235 -3.71151216]
 [-1.10952883 -4.12545979]
 [ 0.46300303  4.76882268]
 [-2.45273835  0.95654184]
 [ 3.11447252 -1.20122944]
 [-3.19510306 -4.36426413]] [(7.2795289742951006, 0, 0, True), (10.492632211674024, 1, 0, True), (3.5837019763659059, 3, 0, True), (8.9543154056833352, 4, 0, True), (8.6155351248604859, 5, 0, True), (3.8069300025933583, 7, 0, True), (7.0432923141042085, 8, 0, True), (9.1136376952482525, 9, 0, True), (4.1773822501669571, 0, 1, True), (4.2703423942086758, 2, 1, True), (9.6996456971854919, 4, 1, True), (7.9908539186183267, 5, 1, True), (2.4307063107290068, 6, 1, True), (4.4100476324521898, 8, 1, True), (5.9423524321239123, 0, 2, True), (8.4430668766249397, 1, 2, True), (0.78659687751450436, 2, 2, True), (3.2473369831522687, 3, 2, True), (6.1613039792